In [30]:
import os
from PIL import Image
from tqdm import tqdm

In [46]:
def add_path(img_name_list: list , path: os.path):
    """
    Combines the complete path to each of the images in a image folder
    
    """
    complete_img_name_list = []
    for i in tqdm(range(len(img_name_list))):
        complete_img_name = os.path.join(path , img_name_list[i])
        # complete_img_name_list.append(complete_img_name)
        img_name_list[i] = complete_img_name

        # print(complete_img_name)
    # return complete_img_name_list
        

In [47]:
dataset_path="data/eyepacs"
subset = "train"

subset_dir = (os.path.join(dataset_path , subset))
        # image dir for that particular subset
img_dir = os.path.join(subset_dir , subset)
# containes all the images name
img_names = os.listdir(img_dir)
add_path(img_name_list=img_names , path=img_dir)

print(img_names[:5])

100%|██████████| 35126/35126 [00:00<00:00, 2322191.58it/s]

['data/eyepacs/train/train/17023_right.jpeg', 'data/eyepacs/train/train/32675_left.jpeg', 'data/eyepacs/train/train/32835_right.jpeg', 'data/eyepacs/train/train/24615_left.jpeg', 'data/eyepacs/train/train/16752_right.jpeg']


In [48]:
import pandas as pd

In [59]:
subset_dir = os.path.join(dataset_path , subset)
subset_files = os.listdir(subset_dir)
label_csv_path = None
for file in subset_files:
    if file.endswith(".csv"):
        label_csv_path = os.path.join(subset_dir , file)


labels_df = pd.read_csv(label_csv_path)
label_dic = {img_name : label for img_name , label in zip(labels_df['image']  , labels_df['level'])} 
#get original image
label_inorder = []
if subset == "train":
    for img_name in img_names:
        label_inorder.append(label_dic[img_name.split('/')[-1].rstrip('.jpeg')])

if subset == "test":
    for img_name in img_names:
        label_inorder.append(label_dic[img_name])

In [64]:
print(label_inorder[-1284])
print(img_names[-1284])

0
data/eyepacs/train/train/26581_left.jpeg


In [58]:
s = "data/eyepacs/train/train/17023_right.jpeg"

print(s.split('/')[-1].rstrip('.jpeg'))

17023_right


In [2]:
l1 = [1,2,3,4]
l2 = [5,7,3,2]
l3 = []
l3.extend(l1)
l3.extend(l2)

print(l3)



[1, 2, 3, 4, 5, 7, 3, 2]


In [ ]:
def BinaryClassificationMetrics(loader: DataLoader, model: nn.Module) -> dict:
    """
    Test your Binary classification model against comprehensive list of metrics.
    
    Args:
        loader (DataLoader): Test DataLoader
        model (nn.Module): Model to be tested
    
    Returns:
        dict: Dictionary containing all calculated metrics
    """
    model.cuda()
    model.eval()
    
    # Initialize counters
    true_positive = 0
    false_positive = 0
    false_negative = 0
    true_negative = 0
    
    # For ROC calculation
    all_probs = []
    all_labels = []
    
    with torch.no_grad():
        for img, label in loader:
            img = img.cuda()
            label = label.cuda()
            
            out = model(img)
            probabilities = torch.softmax(out, dim=1)
            _, pred = out.max(1)
            
            # Store probabilities and labels for ROC calculation
            all_probs.append(probabilities[:, 1].cpu())
            all_labels.append(label.cpu())
            
            # Basic counts
            true_positive += ((pred == 1) & (label == 1)).sum().item()
            false_positive += ((pred == 1) & (label == 0)).sum().item()
            false_negative += ((pred == 0) & (label == 1)).sum().item()
            true_negative += ((pred == 0) & (label == 0)).sum().item()
    
    # Calculate total samples
    num_samples = true_positive + true_negative + false_positive + false_negative
    
    # Basic metrics
    accuracy = ((true_positive + true_negative) / num_samples) * 100
    
    # Handle division by zero cases
    precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0
    recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0
    specificity = true_negative / (true_negative + false_positive) if (true_negative + false_positive) > 0 else 0
    
    # F1 Score
    f1_score = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    # Matthews Correlation Coefficient (MCC)
    numerator = (true_positive * true_negative) - (false_positive * false_negative)
    denominator = sqrt((true_positive + false_positive) * 
                      (true_positive + false_negative) * 
                      (true_negative + false_positive) * 
                      (true_negative + false_negative))
    mcc = numerator / denominator if denominator > 0 else 0
    
    # Kappa Coefficient
    observed_accuracy = (true_positive + true_negative) / num_samples
    expected_accuracy = (((true_positive + false_positive) * 
                         (true_positive + false_negative)) +
                        ((false_negative + true_negative) * 
                         (false_positive + true_negative))) / (num_samples ** 2)
    kappa = (observed_accuracy - expected_accuracy) / (1 - expected_accuracy) if expected_accuracy != 1 else 0
    
    # Calculate ROC-AUC
    try:
        from sklearn.metrics import roc_auc_score
        all_probs = torch.cat(all_probs, dim=0).numpy()
        all_labels = torch.cat(all_labels, dim=0).numpy()
        roc_auc = roc_auc_score(all_labels, all_probs)
    except:
        roc_auc = None
    
    # Return all metrics in a dictionary
    metrics = {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'specificity': specificity,
        'f1_score': f1_score,
        'mcc': mcc,
        'kappa': kappa,
        'roc_auc': roc_auc,
        # Additional useful metrics
        'confusion_matrix': {
            'true_positive': true_positive,
            'false_positive': false_positive,
            'false_negative': false_negative,
            'true_negative': true_negative
        }
    }
    
    return metrics

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
from torchvision import transforms


class MultiCrop(nn.Module):
    """
    
    """

    def __init__(self , encoder , head):
        super(MultiCrop , self).__init__()
        encoder.fc = nn.Identity()
        self.encoder = encoder
        self.head = head

    def forward(self , x):
        crops = torch.cumsum(torch.unique_consecutive(
            torch.tensor([inp.shape[-1] for inp in x]),
            return_counts=True

        )[1] , 0)

        start_index = 0

        for end_index in crops:
            _out = self.backbone(torch.cat(x[start_index : end_index]))
            if start_index == 0:
                output = _out
            
            else:
                output =  torch.cat((output , _out))

            start_index = end_index

        output = self.head(output)
        return output
    
class DINOHead(nn.Module):

    def __init__(self , in_dim , out_dim , hidden_dim=2048 , bottleneck_dim=256):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Linear(in_dim , hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim , hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim , bottleneck_dim),

        )

        self.last_later = nn.Linear(bottleneck_dim , out_dim)
        self.apply(self._init_weights)

    def _init_weights(self , m):
        if isinstance(m , nn.Linear):
            nn.init.trunc_normal_(m.weight , std=0.02)

            if m.bias is not None:
                nn.init.constant_(m.bias , 0)

    def forward(self , x):
        x = self.mlp(x)
        x = F.normalize(x , dim=-1)
        x = self.last_layer(x)
        return x
    

class DINO_loss(nn.Module):

    def __init__(self, out_dim , teach_temp=0.04 , student_temp=0.1 , center_mom=0.9):
        super().__init__()
        self.student_temp = student_temp
        self.teach_temp = teach_temp
        self.center_mom = center_mom

        self.register_buffer("center" , torch.zeros(1, out_dim))

    def forward(self , student_out , teacher_out):
        student_out = student_out / self.student_temp
        teacher_out = teacher_out / self.teach_temp

        student_probs = F.softmax(student_out ,dim=-1)
        teacher_probs = F.softmax((teacher_out - self.center) ,dim=-1).detach()

        loss = torch.sum(-teacher_probs * torch.log_softmax(student_out , dim=-1) , dim=-1).mean()

        self.update_center(teacher_out)

        return loss

    @torch.no_grad()
    def update_center(self , teacher_output):

        batch_center = torch.sum(teacher_output , dim=0 , keepdim=True)
        batch_center = batch_center / len(teacher_output)

        self.center = self.center * self.center_mom + batch_center * (1 - self.center_mom)

class DINO(nn.Module):

    def __init__(self , student , teacher , out_dim , teacher_temp=0.04 , student_temp=0.1 , center_mom=0.9):
        super().__init__()

        self.student = MultiCrop(
            student , 
            DINOHead(student.num_features , out_dim)
        )
        self.teacher = MultiCrop(
            teacher,
            DINOHead(teacher.num_features , out_dim)
        )

        # no gradients for teacher 
        for p in self.teacher.parameters():
            p.requires_grad = False


        self.loss_fn = DINO_loss(
            out_dim=out_dim,
            teacher_temp=teacher_temp,
            student_temp=student_temp,
            center_mom=center_mom,
        )

    @torch.no_grad()
    def update_teacher(self , m=0.996):
        for param_q , param_k in zip(self.student.parameters() , self.teacher.parameters()):
            param_k.data.mul_(m).add_((1-m) * param_q.detach().data)

    def forward(self , x):

        student_output = self.student(x)
        teacher_output = self.teacher(x[:2]) # only the larger global views
        loss = self.loss_fn(student_output , teacher_output)

        return loss



def dino_transforms():
    global_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
        transforms.ToTensor(),
    ])
    
    local_transform = transforms.Compose([
        transforms.RandomResizedCrop(96),
        transforms.RandomHorizontalFlip(),
        transforms.ColorJitter(0.4, 0.4, 0.4, 0.1),
        transforms.ToTensor(),
    ])
    
    return global_transform, local_transform


def train_one_epoch(model , data_loader , optimizer , device , epoch):

    model.train()
    total_loss= 0
    for batch_idx , img in enumerate(data_loader):

        img = [im.to(device) for im in img]

        loss = model(img)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        model.update_teacher()

        total_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f"epoch : {epoch} , batch : {batch_idx} , loss : {loss.item():.2f}")

    return total_loss / len(data_loader)


            

    

         
        
    




        


In [ ]:
import torch
import numpy as np
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
from PIL import Image
import torch.nn.functional as F

class RetinalAugmentationPipeline:
    def __init__(self, 
                 image_size=256,
                 global_crop_size=(224, 224),
                 local_crop_size=(96, 96),
                 num_global_crops=2,
                 num_local_crops=8):
        
        self.num_global_crops = num_global_crops
        self.num_local_crops = num_local_crops
        
        # Basic preprocessing for all views
        self.basic_transform = A.Compose([
            A.Resize(image_size, image_size),
            A.CLAHE(clip_limit=2.0, tile_grid_size=(8, 8), p=1.0),  # Enhance contrast
            A.Normalize(mean=[0.485, 0.456, 0.406], 
                       std=[0.229, 0.224, 0.225]),
        ])
        
        # Global view transformations (stronger)
        self.global_transform = A.Compose([
            A.RandomResizedCrop(height=global_crop_size[0], 
                              width=global_crop_size[1],
                              scale=(0.4, 1.0)),
            A.HorizontalFlip(p=0.5),
            A.RandomRotate90(p=0.5),
            # Color augmentations
            A.OneOf([
                A.RandomBrightnessContrast(
                    brightness_limit=0.2,
                    contrast_limit=0.2,
                    p=1.0
                ),
                A.ColorJitter(
                    brightness=0.2,
                    contrast=0.2,
                    saturation=0.2,
                    hue=0.1,
                    p=1.0
                ),
            ], p=0.8),
            # Blur and noise
            A.OneOf([
                A.GaussianBlur(blur_limit=(3, 7), p=1.0),
                A.GaussNoise(var_limit=(10.0, 50.0), p=1.0),
            ], p=0.5),
            ToTensorV2(),
        ])
        
        # Local view transformations (weaker)
        self.local_transform = A.Compose([
            A.RandomResizedCrop(height=local_crop_size[0], 
                              width=local_crop_size[1],
                              scale=(0.05, 0.4)),
            A.HorizontalFlip(p=0.5),
            # Lighter color augmentations
            A.RandomBrightnessContrast(
                brightness_limit=0.1,
                contrast_limit=0.1,
                p=0.7
            ),
            ToTensorV2(),
        ])

    def preserve_circular_fov(self, image):
        """Preserve circular field of view in fundus images"""
        height, width = image.shape[:2]
        circle_mask = np.zeros((height, width), dtype=np.uint8)
        center = (width // 2, height // 2)
        radius = min(width, height) // 2
        cv2.circle(circle_mask, center, radius, 1, -1)
        
        # Apply mask to each channel
        if len(image.shape) == 3:
            circle_mask = np.stack([circle_mask] * image.shape[2], axis=-1)
        
        return image * circle_mask

    def vessel_enhancement(self, image):
        """Enhance vessel structures using CLAHE"""
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
        l, a, b = cv2.split(lab)
        
        # Apply CLAHE to luminance channel
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        l = clahe.apply(l)
        
        # Merge channels back
        lab = cv2.merge((l, a, b))
        enhanced = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
        return enhanced

    def __call__(self, image, region_masks=None):
        """
        Apply augmentations to create multiple views of the image
        Args:
            image: PIL Image or numpy array
            region_masks: anatomical region masks (optional)
        Returns:
            list of augmented views, transformed region masks
        """
        # Convert PIL Image to numpy if needed
        if isinstance(image, Image.Image):
            image = np.array(image)

        # Basic preprocessing
        preprocessed = self.basic_transform(image=image)['image']
        
        # Preserve circular FOV
        preprocessed = self.preserve_circular_fov(preprocessed)
        
        # Enhance vessel structures
        preprocessed = self.vessel_enhancement(preprocessed)
        
        views = []
        transformed_masks = []
        
        # Create global views
        for _ in range(self.num_global_crops):
            aug = self.global_transform(image=preprocessed)
            views.append(aug['image'])
            
            if region_masks is not None:
                # Apply same transformation to masks
                aug_mask = self.global_transform(image=region_masks)['image']
                transformed_masks.append(aug_mask)
        
        # Create local views
        for _ in range(self.num_local_crops):
            aug = self.local_transform(image=preprocessed)
            views.append(aug['image'])
            
            if region_masks is not None:
                aug_mask = self.local_transform(image=region_masks)['image']
                transformed_masks.append(aug_mask)
        
        return views, transformed_masks if region_masks is not None else views

# Example usage with dataloader
class RetinalDataset(torch.utils.data.Dataset):
    def __init__(self, image_paths, mask_paths=None, transform=None):
        self.image_paths = image_paths
        self.mask_paths = mask_paths
        self.transform = transform or RetinalAugmentationPipeline()
    
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        
        if self.mask_paths is not None:
            mask = Image.open(self.mask_paths[idx])
            views, transformed_masks = self.transform(image, mask)
            return views, transformed_masks
        else:
            views = self.transform(image)
            return views
    
    def __len__(self):
        return len(self.image_paths)

In [3]:
l = [1,2,3,4,5]
m = ["a" , "b"  ,"c" , "d" , "e"]

pa = list(zip(m , l))
import random

random.shuffle(pa)

m , l = map(list , zip(*pa))

print(m)
print(l)

['e', 'b', 'd', 'c', 'a']
[5, 2, 4, 3, 1]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from einops import rearrange

class PatchEmbedding(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_chans=3, embed_dim=768):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2
        
        self.proj = nn.Conv2d(
            in_chans, embed_dim,
            kernel_size=patch_size, stride=patch_size
        )

    def forward(self, x):
        x = self.proj(x)  # (B, E, H', W')
        x = rearrange(x, 'b e h w -> b (h w) e')
        return x

class TransformerEncoder(nn.Module):
    def __init__(self, dim, depth, heads, mlp_dim, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                nn.LayerNorm(dim),
                nn.MultiheadAttention(dim, heads, dropout=dropout),
                nn.LayerNorm(dim),
                nn.Sequential(
                    nn.Linear(dim, mlp_dim),
                    nn.GELU(),
                    nn.Dropout(dropout),
                    nn.Linear(mlp_dim, dim),
                    nn.Dropout(dropout)
                )
            ]))

    def forward(self, x):
        for norm1, attn, norm2, mlp in self.layers:
            x_norm = norm1(x)
            attn_out, _ = attn(x_norm, x_norm, x_norm)
            x = x + attn_out
            x = x + mlp(norm2(x))
        return x

class IJEPA(nn.Module):
    def __init__(
        self,
        img_size=224,
        patch_size=16,
        in_chans=3,
        embed_dim=768,
        encoder_depth=12,
        predictor_depth=4,
        num_heads=12,
        mlp_ratio=4,
        dropout=0.1
    ):
        super().__init__()
        
        # Image embedding
        self.patch_embed = PatchEmbedding(img_size, patch_size, in_chans, embed_dim)
        
        # Target encoder
        self.target_encoder = TransformerEncoder(
            embed_dim,
            encoder_depth,
            num_heads,
            embed_dim * mlp_ratio,
            dropout
        )
        
        # Context encoder
        self.context_encoder = TransformerEncoder(
            embed_dim,
            encoder_depth,
            num_heads,
            embed_dim * mlp_ratio,
            dropout
        )
        
        # Predictor
        self.predictor = TransformerEncoder(
            embed_dim,
            predictor_depth,
            num_heads,
            embed_dim * mlp_ratio,
            dropout
        )
        
        # Initialize weights
        self.apply(self._init_weights)

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    @torch.no_grad()
    def momentum_update(self, target_encoder, context_encoder, momentum=0.999):
        """Momentum update for target encoder"""
        for target_params, context_params in zip(
            target_encoder.parameters(), context_encoder.parameters()
        ):
            target_params.data = momentum * target_params.data + (1 - momentum) * context_params.data

    def get_random_boxes(self, batch_size, n_boxes=4):
        """Generate random target boxes for masked prediction"""
        boxes = []
        for _ in range(batch_size):
            batch_boxes = []
            for _ in range(n_boxes):
                x1 = torch.randint(0, 14, (1,)).item()
                y1 = torch.randint(0, 14, (1,)).item()
                w = torch.randint(2, 6, (1,)).item()
                h = torch.randint(2, 6, (1,)).item()
                batch_boxes.append([x1, y1, x1 + w, y1 + h])
            boxes.append(batch_boxes)
        return torch.tensor(boxes)

    def extract_targets(self, features, boxes):
        """Extract target features based on boxes"""
        B, N, D = features.shape
        H = W = int(N ** 0.5)
        features = rearrange(features, 'b (h w) d -> b d h w', h=H)
        
        target_features = []
        for b in range(B):
            batch_targets = []
            for box in boxes[b]:
                x1, y1, x2, y2 = box
                target = features[b:b+1, :, y1:y2, x1:x2]
                target = F.adaptive_avg_pool2d(target, (1, 1)).squeeze(-1).squeeze(-1)
                batch_targets.append(target)
            target_features.append(torch.cat(batch_targets, dim=0))
        return torch.stack(target_features)

    def forward(self, images, boxes=None):
        B = images.shape[0]
        if boxes is None:
            boxes = self.get_random_boxes(B)
            
        # Get patch embeddings
        x = self.patch_embed(images)
        
        # Get context features
        context_features = self.context_encoder(x)
        
        # Get target features (with no gradient)
        with torch.no_grad():
            target_features = self.target_encoder(x)
            target_features = self.extract_targets(target_features, boxes)
        
        # Predict target features
        predicted_features = self.predictor(context_features)
        predicted_features = self.extract_targets(predicted_features, boxes)
        
        return predicted_features, target_features

class IJEPALoss(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, predicted_features, target_features):
        # Normalize features
        predicted_features = F.normalize(predicted_features, dim=-1)
        target_features = F.normalize(target_features, dim=-1)
        
        # Compute cosine similarity
        sim = torch.einsum('bnd,bnd->bn', predicted_features, target_features)
        
        # Compute loss (negative cosine similarity)
        loss = -sim.mean()
        
        return loss

def create_ijepa_model(
    img_size=224,
    patch_size=16,
    in_chans=3,
    embed_dim=768,
    encoder_depth=12,
    predictor_depth=4,
    num_heads=12,
    mlp_ratio=4,
    dropout=0.1
):
    model = IJEPA(
        img_size=img_size,
        patch_size=patch_size,
        in_chans=in_chans,
        embed_dim=embed_dim,
        encoder_depth=encoder_depth,
        predictor_depth=predictor_depth,
        num_heads=num_heads,
        mlp_ratio=mlp_ratio,
        dropout=dropout
    )
    criterion = IJEPALoss()
    return model, criterion

# DR specific IJEP

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from einops import rearrange

class PatchEmbedding(nn.Module):
    def __init__(self, img_size=2048, patch_size=32, in_chans=3, embed_dim=1024):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2
        
        # Hierarchical patch embedding to handle large images
        self.hierarchical_proj = nn.Sequential(
            nn.Conv2d(in_chans, embed_dim//4, kernel_size=7, stride=2, padding=3),
            nn.LayerNorm([embed_dim//4, img_size//2, img_size//2]),
            nn.GELU(),
            nn.Conv2d(embed_dim//4, embed_dim//2, kernel_size=3, stride=2, padding=1),
            nn.LayerNorm([embed_dim//2, img_size//4, img_size//4]),
            nn.GELU(),
            nn.Conv2d(embed_dim//2, embed_dim, kernel_size=patch_size//4, stride=patch_size//4)
        )

    def forward(self, x):
        x = self.hierarchical_proj(x)  # (B, E, H', W')
        x = rearrange(x, 'b e h w -> b (h w) e')
        return x

class TransformerEncoder(nn.Module):
    def __init__(self, dim, depth, heads, mlp_dim, dropout=0.1):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            self.layers.append(nn.ModuleList([
                nn.LayerNorm(dim),
                nn.MultiheadAttention(dim, heads, dropout=dropout),
                nn.LayerNorm(dim),
                nn.Sequential(
                    nn.Linear(dim, mlp_dim),
                    nn.GELU(),
                    nn.Dropout(dropout),
                    nn.Linear(mlp_dim, dim),
                    nn.Dropout(dropout)
                )
            ]))

    def forward(self, x):
        for norm1, attn, norm2, mlp in self.layers:
            x_norm = norm1(x)
            attn_out, _ = attn(x_norm, x_norm, x_norm)
            x = x + attn_out
            x = x + mlp(norm2(x))
        return x

class MedicalIJEPA(nn.Module):
    def __init__(
        self,
        img_size=2048,
        patch_size=32,  # Larger patch size for memory efficiency
        in_chans=3,
        embed_dim=1024,  # Increased embedding dimension
        encoder_depth=12,
        predictor_depth=4,
        num_heads=16,  # Increased number of heads
        mlp_ratio=4,
        dropout=0.1
    ):
        super().__init__()
        
        self.patch_embed = PatchEmbedding(img_size, patch_size, in_chans, embed_dim)
        
        # Target encoder
        self.target_encoder = TransformerEncoder(
            embed_dim,
            encoder_depth,
            num_heads,
            embed_dim * mlp_ratio,
            dropout
        )
        
        # Context encoder
        self.context_encoder = TransformerEncoder(
            embed_dim,
            encoder_depth,
            num_heads,
            embed_dim * mlp_ratio,
            dropout
        )
        
        # Predictor with medical-specific adjustments
        self.predictor = nn.Sequential(
            TransformerEncoder(
                embed_dim,
                predictor_depth,
                num_heads,
                embed_dim * mlp_ratio,
                dropout
            ),
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, embed_dim)  # Additional projection layer
        )
        
        # Initialize weights
        self.apply(self._init_weights)
        
        # Save grid size for box generation
        self.grid_size = img_size // patch_size

    def _init_weights(self, m):
        if isinstance(m, nn.Linear):
            torch.nn.init.xavier_uniform_(m.weight)
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.LayerNorm):
            nn.init.constant_(m.bias, 0)
            nn.init.constant_(m.weight, 1.0)

    def get_random_boxes(self, batch_size, n_boxes=6):  # Increased number of boxes
        """Generate random target boxes with medical imaging considerations"""
        boxes = []
        for _ in range(batch_size):
            batch_boxes = []
            # Ensure better coverage of the retinal area
            for _ in range(n_boxes):
                # Generate boxes more likely to be in the central region
                center_bias = torch.randn(2) * 0.5
                x_center = self.grid_size // 2 + int(center_bias[0] * self.grid_size // 4)
                y_center = self.grid_size // 2 + int(center_bias[1] * self.grid_size // 4)
                
                # Random box size (larger for better feature capture)
                w = torch.randint(4, 8, (1,)).item()
                h = torch.randint(4, 8, (1,)).item()
                
                x1 = max(0, x_center - w // 2)
                y1 = max(0, y_center - h // 2)
                x2 = min(self.grid_size, x1 + w)
                y2 = min(self.grid_size, y1 + h)
                
                batch_boxes.append([x1, y1, x2, y2])
            boxes.append(batch_boxes)
        return torch.tensor(boxes)

    def forward(self, images, boxes=None):
        B = images.shape[0]
        if boxes is None:
            boxes = self.get_random_boxes(B)
            
        # Get patch embeddings
        x = self.patch_embed(images)
        
        # Get context features
        context_features = self.context_encoder(x)
        
        # Get target features (with no gradient)
        with torch.no_grad():
            target_features = self.target_encoder(x)
            target_features = self.extract_targets(target_features, boxes)
        
        # Predict target features
        predicted_features = self.predictor(context_features)
        predicted_features = self.extract_targets(predicted_features, boxes)
        
        return predicted_features, target_features

    def extract_targets(self, features, boxes):
        """Extract target features with medical imaging considerations"""
        B, N, D = features.shape
        H = W = int(N ** 0.5)
        features = rearrange(features, 'b (h w) d -> b d h w', h=H)
        
        target_features = []
        for b in range(B):
            batch_targets = []
            for box in boxes[b]:
                x1, y1, x2, y2 = box
                target = features[b:b+1, :, y1:y2, x1:x2]
                # Use adaptive pooling for variable size boxes
                target = F.adaptive_avg_pool2d(target, (2, 2))
                target = rearrange(target, 'b c h w -> b (h w) c')
                batch_targets.append(target)
            target_features.append(torch.cat(batch_targets, dim=1))
        return torch.stack(target_features)

def create_medical_ijepa(
    img_size=2048,
    patch_size=32,
    in_chans=3,
    embed_dim=1024,
    encoder_depth=12,
    predictor_depth=4,
    num_heads=16,
    mlp_ratio=4,
    dropout=0.1
):
    model = MedicalIJEPA(
        img_size=img_size,
        patch_size=patch_size,
        in_chans=in_chans,
        embed_dim=embed_dim,
        encoder_depth=encoder_depth,
        predictor_depth=predictor_depth,
        num_heads=num_heads,
        mlp_ratio=mlp_ratio,
        dropout=dropout
    )
    criterion = nn.MSELoss()  # Using MSE loss for medical imaging
    return model, criterion

# Medical image specific transforms
def get_medical_transforms(img_size=2048):
    train_transforms = transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
        transforms.RandomRotation(30),
        transforms.ColorJitter(brightness=0.1, contrast=0.1),
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    val_transforms = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return train_transforms, val_transforms

# DR specific IJEPA with adv

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from einops import rearrange
import numpy as np
from torch.cuda.amp import autocast, GradScaler
import albumentations as A

class DRSpecificAugmentation:
    def __init__(self):
        self.transform = A.Compose([
            # DR-specific augmentations
            A.RandomBrightnessContrast(
                brightness_limit=0.1, 
                contrast_limit=0.1, 
                p=0.5
            ),
            A.OneOf([
                # Simulate different lighting conditions
                A.GaussNoise(var_limit=(10.0, 50.0), p=0.5),
                A.MultiplicativeNoise(multiplier=(0.95, 1.05), p=0.5),
            ], p=0.3),
            A.OneOf([
                # Simulate different imaging conditions
                A.GaussianBlur(blur_limit=(3, 5), p=0.3),
                A.MedianBlur(blur_limit=3, p=0.3),
            ], p=0.2),
            # Lesion-focused augmentations
            A.OneOf([
                A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.5),
                A.UnsharpMask(blur_limit=(3, 7), p=0.5),
            ], p=0.3),
            # Color augmentations for different camera settings
            A.HueSaturationValue(
                hue_shift_limit=5,
                sat_shift_limit=10,
                val_shift_limit=5,
                p=0.3
            ),
            # Preserve circular FOV
            A.CoarseDropout(
                max_holes=8,
                max_height=32,
                max_width=32,
                min_holes=1,
                min_height=8,
                min_width=8,
                fill_value=0,
                mask_fill_value=0,
                p=0.2
            ),
        ])

    def __call__(self, image):
        return self.transform(image=image)['image']

class MemoryEfficientBatchSampler:
    def __init__(self, dataset, batch_size, shuffle=True):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.gradient_accumulation_steps = 4  # Adjust based on memory constraints
        
    def __iter__(self):
        if self.shuffle:
            indices = torch.randperm(len(self.dataset))
        else:
            indices = torch.arange(len(self.dataset))
            
        # Create mini-batches
        mini_batch_size = self.batch_size // self.gradient_accumulation_steps
        for i in range(0, len(indices), mini_batch_size):
            yield indices[i:i + mini_batch_size]

class DRLesionAttention(nn.Module):
    """Attention module specifically designed for DR lesions"""
    def __init__(self, dim):
        super().__init__()
        self.scale = dim ** -0.5
        self.lesion_queries = nn.Parameter(torch.randn(1, 5, dim))  # 5 different lesion types
        self.to_qkv = nn.Linear(dim, dim * 3, bias=False)
        
    def forward(self, x):
        b, n, d = x.shape
        qkv = self.to_qkv(x).chunk(3, dim=-1)
        q, k, v = map(lambda t: rearrange(t, 'b n d -> b n d'), qkv)
        
        # Add lesion-specific queries
        lesion_queries = self.lesion_queries.repeat(b, 1, 1)
        q = torch.cat([q, lesion_queries], dim=1)
        
        dots = torch.matmul(q, k.transpose(-1, -2)) * self.scale
        attn = dots.softmax(dim=-1)
        out = torch.matmul(attn, v)
        
        return out

class DRSpecificIJEPA(MedicalIJEPA):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # Add DR-specific components
        self.lesion_attention = DRLesionAttention(kwargs.get('embed_dim', 1024))
        
        # Add lesion-specific prediction heads
        self.lesion_heads = nn.ModuleDict({
            'microaneurysms': nn.Linear(kwargs.get('embed_dim', 1024), 1),
            'hemorrhages': nn.Linear(kwargs.get('embed_dim', 1024), 1),
            'hard_exudates': nn.Linear(kwargs.get('embed_dim', 1024), 1),
            'cotton_wool_spots': nn.Linear(kwargs.get('embed_dim', 1024), 1),
            'neovascularization': nn.Linear(kwargs.get('embed_dim', 1024), 1)
        })

    def forward(self, images, boxes=None):
        features, target_features = super().forward(images, boxes)
        
        # Apply lesion attention
        features_with_attention = self.lesion_attention(features)
        
        # Get lesion-specific predictions
        lesion_predictions = {
            name: head(features_with_attention.mean(1))
            for name, head in self.lesion_heads.items()
        }
        
        return features, target_features, lesion_predictions

class DRTrainer:
    def __init__(
        self,
        model,
        criterion,
        optimizer,
        device,
        grad_accumulation_steps=4,
        mixed_precision=True
    ):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.device = device
        self.grad_accumulation_steps = grad_accumulation_steps
        self.mixed_precision = mixed_precision
        self.scaler = GradScaler() if mixed_precision else None
        
    def train_step(self, batch, lesion_labels=None):
        self.model.train()
        total_loss = 0
        
        # Split batch for gradient accumulation
        micro_batches = torch.chunk(batch, self.grad_accumulation_steps)
        
        for i, micro_batch in enumerate(micro_batches):
            with autocast(enabled=self.mixed_precision):
                features, target_features, lesion_preds = self.model(micro_batch)
                
                # Compute main I-JEPA loss
                main_loss = self.criterion(features, target_features)
                
                # Compute lesion prediction loss if labels are provided
                lesion_loss = 0
                if lesion_labels is not None:
                    for name, pred in lesion_preds.items():
                        lesion_loss += F.binary_cross_entropy_with_logits(
                            pred,
                            lesion_labels[name][i * len(micro_batch):(i + 1) * len(micro_batch)]
                        )
                
                loss = main_loss + 0.5 * lesion_loss
                scaled_loss = loss / self.grad_accumulation_steps
                
            if self.mixed_precision:
                self.scaler.scale(scaled_loss).backward()
            else:
                scaled_loss.backward()
            
            total_loss += loss.item()
        
        # Update weights
        if self.mixed_precision:
            self.scaler.step(self.optimizer)
            self.scaler.update()
        else:
            self.optimizer.step()
            
        self.optimizer.zero_grad()
        
        return total_loss / self.grad_accumulation_steps

def create_dr_specific_transforms(img_size=2048):
    """Create DR-specific image transformations"""
    train_transforms = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        DRSpecificAugmentation(),
    ])
    
    val_transforms = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    return train_transforms, val_transforms

def train_dr_model(
    model,
    train_loader,
    val_loader,
    num_epochs,
    device,
    learning_rate=1e-4,
    weight_decay=1e-4
):
    """Training loop with DR-specific optimizations"""
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=learning_rate,
        weight_decay=weight_decay
    )
    
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=num_epochs
    )
    
    trainer = DRTrainer(
        model=model,
        criterion=nn.MSELoss(),
        optimizer=optimizer,
        device=device,
        mixed_precision=True
    )
    
    for epoch in range(num_epochs):
        train_loss = 0
        val_loss = 0
        
        # Training
        model.train()
        for batch, lesion_labels in train_loader:
            batch = batch.to(device)
            loss = trainer.train_step(batch, lesion_labels)
            train_loss += loss
            
        # Validation
        model.eval()
        with torch.no_grad():
            for batch, lesion_labels in val_loader:
                batch = batch.to(device)
                features, target_features, lesion_preds = model(batch)
                loss = trainer.criterion(features, target_features)
                val_loss += loss.item()
        
        scheduler.step()
        
        print(f"Epoch {epoch+1}/{num_epochs}")
        print(f"Train Loss: {train_loss/len(train_loader):.4f}")
        print(f"Val Loss: {val_loss/len(val_loader):.4f}")